This notebook is designed to recalculate the metrics based on only the manual picks

In [56]:
import pandas as pd

# 1. Load data

In [57]:
reference_phase=pd.read_csv("/Users/ziyixi/Library/CloudStorage/OneDrive-MichiganStateUniversity/Paper/PhaseNetTF_myturn/PhaseNet-TF-Figures/phasenettf/data/catalog/tonga_picks_updated_2023_0426.csv")
reference_phase_melted = reference_phase.melt(id_vars=['originid', 'stacode'], 
                    value_vars=['arrival_time_P', 'arrival_time_S'], 
                    var_name='type', 
                    value_name='time')
reference_phase_melted['type'] = reference_phase_melted['type'].str.replace('arrival_time_', '')
reference_phase_melted["type"]=reference_phase_melted["type"].replace({"P":"p", "S":"s"})
reference_phase_melted["time"]=pd.to_datetime(reference_phase_melted["time"])
reference_phase_melted.rename(columns={"stacode":"sta", "originid":"mp_event_index"}, inplace=True)
reference_phase_melted.dropna(inplace=True)
reference_phase=reference_phase_melted
reference_phase.head(),len(reference_phase)

(  mp_event_index   sta type                       time
 0       11_52111   A01    p 2009-12-02 01:14:41.317870
 1       11_52111  A02W    p 2009-12-02 01:14:42.457750
 2       11_52111   A03    p 2009-12-02 01:14:43.547000
 3       11_52111   A05    p 2009-12-02 01:14:47.038000
 4       11_52111  A06W    p 2009-12-02 01:14:48.857230,
 57108)

In [58]:
reference_catalog=pd.read_csv("/Users/ziyixi/Library/CloudStorage/OneDrive-MichiganStateUniversity/Paper/PhaseNetTF_myturn/PhaseNet-TF-Figures/phasenettf/data/catalog/tonga_catalog_updated_2023_0426.csv")
reference_catalog.drop(columns=["eventid","author","subset","mb","ISCid"],inplace=True)
reference_catalog["time"]=pd.to_datetime(reference_catalog["time"])
reference_catalog.rename(columns={"originid":"mp_event_index"},inplace=True)
reference_catalog.head(),len(reference_catalog)

(   latitude  longitude     depth                       time mp_event_index
 0  -21.4486  -175.0123   59.8167 2009-12-02 01:14:28.278360       11_52111
 1  -21.0021  -176.0597  189.5062 2009-12-02 02:25:13.607840       11_52113
 2  -20.0291  -175.3772  179.2744 2009-12-02 15:41:32.776970       11_52115
 3  -20.8052  -174.5321   66.1092 2009-12-03 09:43:16.236000       11_52119
 4  -19.4698  -174.9927  146.2841 2009-12-04 17:37:18.418720       11_52124,
 1163)

## 1.1 association

In [59]:
iteration1_phasenettf_phase=pd.read_csv(
        "/Users/ziyixi/Library/CloudStorage/OneDrive-MichiganStateUniversity/Paper/PhaseNetTF_myturn/PhaseNet-TF-Figures/phasenettf/data/catalog/manual_picks/manual_phase_picks.assignments.csv",
        skiprows=1,
        names=[
            "id",
            "date",
            "time",
            "amp",
            "type",
            "prob",
            "event_index",
            "gamma_score",
        ],
        sep=r"\s+",
    )

iteration1_phasenettf_phase["sta"]=iteration1_phasenettf_phase["id"].str.split(".").str[1]
iteration1_phasenettf_phase["time"]=pd.to_datetime(iteration1_phasenettf_phase["date"]+" "+iteration1_phasenettf_phase["time"])
iteration1_phasenettf_phase=iteration1_phasenettf_phase.drop(["date","id","amp","prob"],axis=1)
iteration1_phasenettf_phase.head(),len(iteration1_phasenettf_phase)

(                               time type  event_index  gamma_score   sta
 11_52111 2009-12-02 01:14:41.317870    p         1052     0.213809   A01
 11_52111 2009-12-02 01:14:51.343700    s         1052     0.213613   A01
 11_52111 2009-12-02 01:14:42.457750    p         1052     0.196280  A02W
 11_52111 2009-12-02 01:14:53.596650    s         1052     0.202071  A02W
 11_52111 2009-12-02 01:14:43.547000    p         1052     0.200115   A03,
 55183)

In [60]:
iteration1_association_catalog = pd.read_csv(
        "/Users/ziyixi/Library/CloudStorage/OneDrive-MichiganStateUniversity/Paper/PhaseNetTF_myturn/PhaseNet-TF-Figures/phasenettf/data/catalog/manual_picks/manual_phase_picks.catalog.csv",
        usecols=["time", "longitude", "latitude", "z(km)", "event_index"],
        sep=r"\s+",
    )
iteration1_association_catalog["time"]=pd.to_datetime(iteration1_association_catalog["time"])
iteration1_association_catalog.rename(
        columns={
            "z(km)": "depth",
        },
        inplace=True,
    )
iteration1_association_catalog = iteration1_association_catalog[
        iteration1_association_catalog["event_index"].isin(
            iteration1_phasenettf_phase["event_index"]
            .value_counts()[iteration1_phasenettf_phase["event_index"].value_counts() >= 10]
            .index
        )
    ]
iteration1_association_catalog.head(),len(iteration1_association_catalog)

(                     time       depth  event_index   longitude   latitude
 0 2010-04-05 14:35:58.504  211.315330           31 -175.605850 -20.355755
 1 2010-03-01 08:48:42.766  531.988000          144 -178.226265 -17.511445
 2 2010-03-01 08:48:46.110  522.842526          145 -177.679249 -17.689561
 3 2009-12-08 12:39:07.872  208.602000           65 -176.107451 -21.210745
 4 2010-04-17 01:37:08.645  507.835920           73 -177.392737 -20.200993,
 1187)

In [61]:
# select iteration1_association_phase from iteration1_phasenettf_phase with event_index in iteration1_association_catalog
iteration1_association_catalog_event_index_set=set(iteration1_association_catalog["event_index"])
iteration1_association_phase=iteration1_phasenettf_phase[iteration1_phasenettf_phase["event_index"].isin(iteration1_association_catalog_event_index_set)]
iteration1_association_phase.head(),len(iteration1_association_phase)

(                               time type  event_index  gamma_score   sta
 11_52111 2009-12-02 01:14:41.317870    p         1052     0.213809   A01
 11_52111 2009-12-02 01:14:51.343700    s         1052     0.213613   A01
 11_52111 2009-12-02 01:14:42.457750    p         1052     0.196280  A02W
 11_52111 2009-12-02 01:14:53.596650    s         1052     0.202071  A02W
 11_52111 2009-12-02 01:14:43.547000    p         1052     0.200115   A03,
 53332)

## 1.2 Relocation

In [62]:
iteration1_relocation_catalog = pd.read_csv(
        "/Users/ziyixi/Library/CloudStorage/OneDrive-MichiganStateUniversity/Paper/PhaseNetTF_myturn/PhaseNet-TF-Figures/phasenettf/data/catalog/manual_picks/bootstrap.teletomodd.csv",
        usecols=[0, 1, 2, 3, 4],
    )
iteration1_relocation_catalog.rename(
        columns={
            "id": "event_index",
        },
        inplace=True,
    )
iteration1_relocation_catalog["time"] = pd.to_datetime(iteration1_relocation_catalog["datetime"],format='ISO8601')
iteration1_relocation_catalog.drop(columns=["datetime"], inplace=True)
iteration1_relocation_catalog.head(),len(iteration1_relocation_catalog)

(   event_index   latitude    lonitude       depth  \
 0            0 -17.359103 -178.506035  668.547502   
 1            2 -17.073254 -173.492853   27.456746   
 2            3 -18.039941 -177.789039  426.305508   
 3            4 -21.547365 -178.622186  660.387356   
 4            5 -20.881543 -176.394364  187.810662   
 
                            time  
 0 2010-03-04 23:45:09.862120192  
 1 2010-02-24 07:46:01.778779904  
 2 2010-05-20 02:23:24.466560000  
 3 2010-03-12 12:44:05.611340032  
 4 2010-01-12 11:04:22.847159808  ,
 1158)

In [63]:
# select iteration1_relocation_phase from iteration1_phasenettf_phase with event_index in iteration1_relocation_catalog
iteration1_relocation_catalog_event_index_set=set(iteration1_relocation_catalog["event_index"])
iteration1_relocation_phase=iteration1_phasenettf_phase[iteration1_phasenettf_phase["event_index"].isin(iteration1_relocation_catalog_event_index_set)]
iteration1_relocation_phase.head(),len(iteration1_relocation_phase)

(                               time type  event_index  gamma_score   sta
 11_52111 2009-12-02 01:14:41.317870    p         1052     0.213809   A01
 11_52111 2009-12-02 01:14:51.343700    s         1052     0.213613   A01
 11_52111 2009-12-02 01:14:42.457750    p         1052     0.196280  A02W
 11_52111 2009-12-02 01:14:53.596650    s         1052     0.202071  A02W
 11_52111 2009-12-02 01:14:43.547000    p         1052     0.200115   A03,
 52686)

# Analysis

In [64]:
def calculate_event_based_metrics(df_new_raw:pd.DataFrame,df_ref_raw:pd.DataFrame):
    # calculate precision, recall, and f1-score, example df_new: iteration1_association_catalog, example df_ref: reference_catalog
    df_new=df_new_raw.copy()
    df_ref=df_ref_raw.copy()
    df_new.sort_values(by=["time"],inplace=True)
    df_ref.sort_values(by=["time"],inplace=True)
    
    merged=pd.merge_asof(df_new,df_ref,on="time",direction="nearest",tolerance=pd.Timedelta("15s"))
    merged.dropna(inplace=True)
    merged.drop_duplicates(subset=["mp_event_index"],inplace=True)
    # calculate precision
    tp=len(merged)
    fp=len(df_new)-len(merged)
    fn=len(df_ref)-len(merged)
    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    f1_score=2*precision*recall/(precision+recall)
    return {
        "tp":tp,
        "fp":fp,
        "fn":fn,
        "precision":precision,
        "recall":recall,
        "f1_score":f1_score,
    }

In [65]:
calculate_event_based_metrics(iteration1_association_catalog,reference_catalog)

{'tp': 1123,
 'fp': 64,
 'fn': 40,
 'precision': 0.9460825610783488,
 'recall': 0.9656061908856406,
 'f1_score': 0.9557446808510639}

In [66]:
calculate_event_based_metrics(iteration1_relocation_catalog,reference_catalog)

{'tp': 1103,
 'fp': 55,
 'fn': 60,
 'precision': 0.9525043177892919,
 'recall': 0.9484092863284609,
 'f1_score': 0.9504523912106851}

In [67]:
def calculate_phase_based_metrics(df_new_raw:pd.DataFrame,df_ref_raw:pd.DataFrame,phase:str):
    # calculate precision, recall, and f1-score, example df_new: iteration1_association_phase, example df_ref: reference_phase
    df_new=df_new_raw[df_new_raw["type"]==phase].copy()
    df_ref=df_ref_raw[df_ref_raw["type"]==phase].copy()
    df_new.sort_values(by=["time","sta"],inplace=True)
    df_ref.sort_values(by=["time","sta"],inplace=True)
    
    merged=pd.merge_asof(df_new,df_ref,on="time",by="sta",direction="nearest",tolerance=pd.Timedelta("0.5s"))
    merged.dropna(inplace=True)
    merged.drop_duplicates(subset=["mp_event_index","sta"],inplace=True)
    # calculate precision
    tp=len(merged)
    fp=len(df_new)-len(merged)
    fn=len(df_ref)-len(merged)
    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    f1_score=2*precision*recall/(precision+recall)
    return {
        "tp":tp,
        "fp":fp,
        "fn":fn,
        "precision":precision,
        "recall":recall,
        "f1_score":f1_score,
    }

In [68]:
calculate_phase_based_metrics(iteration1_association_phase,reference_phase,"p")

{'tp': 40481,
 'fp': 0,
 'fn': 1775,
 'precision': 1.0,
 'recall': 0.9579941310109807,
 'f1_score': 0.9785464786008679}

In [69]:
calculate_phase_based_metrics(iteration1_association_phase,reference_phase,"s")

{'tp': 12851,
 'fp': 0,
 'fn': 2001,
 'precision': 1.0,
 'recall': 0.865270670616752,
 'f1_score': 0.927769555643793}

In [70]:
calculate_phase_based_metrics(iteration1_relocation_phase,reference_phase,"p")

{'tp': 39969,
 'fp': 0,
 'fn': 2287,
 'precision': 1.0,
 'recall': 0.9458775085195001,
 'f1_score': 0.9721860747947704}

In [71]:
calculate_phase_based_metrics(iteration1_relocation_phase,reference_phase,"s")

{'tp': 12717,
 'fp': 0,
 'fn': 2135,
 'precision': 1.0,
 'recall': 0.8562483167250202,
 'f1_score': 0.9225579455185171}